In [16]:
import folium
from geopy.geocoders import Nominatim
import pandas as pd
import requests
import time
from folium.features import CustomIcon



In [32]:
try:
    df = pd.read_csv("kids/chibakidscafelist.csv", encoding = "cp932")
except FileNotFoundError:
    print("指定されたファイルが見つかりません。ファイル名を確認してください。")
    exit()


# 誤入力であろう?を-に変換する
df = df.astype(str).replace({r'\?': '-'}, regex=True)

#改行処理を無効に
df = df.replace("\n", "", regex = True)

#lat列とlong列を作成
if "lat" not in df.columns:
    df["lat"] = None
if "long" not in df.columns:
    df["long"] = None

address_col = "所在地"

#住所から緯度経度情報を取得する関数
def lat_long(address):
    url = f"https://msearch.gsi.go.jp/address-search/AddressSearch?q={address}"
    try:
        res = requests.get(url)
        data = res.json()
        if len(data) > 0:
            return data[0]["geometry"]["coordinates"][1], data[0]["geometry"]["coordinates"][0]
        else:
            return None, None
    except:
        return None, None
        
#緯度経度情報がない値を取得
missing_rows = df[df[["lat", "long"]].isnull().any(axis=1)]

#関数lat_longから緯度経度を取得する
for i, addr in missing_rows[address_col].items():
    lat, long = lat_long(addr)
    df.at[i, "lat"] = lat
    df.at[i, "long"] = long

#データがない場合は - で表示
df = df.fillna("-")

#CSVファイルに保存
df.to_csv("kids/千葉市子ども食堂一覧_copy.csv", index = False, encoding="cp932")

In [18]:
#今日の日付を入力
today = datetime.date.today()

#今日が第何週の何曜日かを2桁のコードで返す
def get_weekday(date):

    weekday = date.weekday()+1
    week_num = (date.day-1) // 7 + 1

    code = (week_num - 1) * 7 + weekday
    return str(code).zfill(2)



def open_today(code_str, date=today):
    if pd.isna(code_str):
        return False
        
    #偶数月の判別
    if "EV" in code_str and date.month % 2 == 1:
        return False

    today_code = get_weekday(date)
    codes = [code_str[i:i+2] for i in range(0, len(code_str), 2)]
    return today_code in codes

print(get_weekday(today))
df["営業中"] = df["日時"].apply(lambda x: open_today(str(x), today))

print(df[["こども食堂の名称", "開催日", "営業中"]])

08
            こども食堂の名称                        開催日    営業中
0          きぼーるこども食堂  毎月1回（曜日は未定）6月～9月は開催いたしません  False
1        こども食堂 やまちゃん               夏休み限定（週1～2回）  False
2      グレイステーブルこども食堂     毎月1回土曜日学校の長期休暇期間は不定期開催  False
3           松ケ丘子ども食堂                        年数回  False
4              こども食堂                    毎月第1火曜日  False
5          ラヴィーレ千葉椿森                    毎月第4土曜日  False
6        松波フレンズこども食堂                      第３土曜日  False
7           幸福軒子ども食堂                    毎月第３木曜日  False
8       おおなみこなみ「青空市」                      年３回程度  False
9          西小中台子ども食堂                    毎月第4土曜日  False
10          和らぎ子ども食堂             毎月第2日曜日（休日開庁日）  False
11            はたまち食堂                    毎月第3金曜日  False
12           柏井こども食堂              第2木曜日夕方第4日曜日昼  False
13         そんぽの家朝日ヶ丘                    毎月第4土曜日  False
14            あったカフェ                    毎月第１土曜日  False
15     花園みんなのカフェTOMO               毎週火曜日毎月第1土曜日  False
16           畑町子ども食堂                    毎月第３日曜日  False
17     

In [20]:

# ジオコーダー
geolocator = Nominatim(user_agent="my_map_app")

# 地図の中心座標（適当に最初の住所）
chiba_center = [35.6076, 140.1063]
# 千葉市のおおよその境界（緯度経度）
sw = [35.50, 140.05]  # 南西端
ne = [35.70, 140.20]  # 北東端

m = folium.Map(
    location=chiba_center,
    zoom_start=13,
    tiles='https://cyberjapandata.gsi.go.jp/xyz/pale/{z}/{x}/{y}.png',
    attr='国土地理院',
    min_zoom=12,
    max_zoom=18,
    max_bounds=True
)
m.fit_bounds([sw, ne])
# 住所を緯度経度に変換してピンを追加
for _, place in df.iterrows():
    if pd.notna(place["lat"]) and pd.notna(place["long"]):

        popup_html = f"""
        <div style="white-space:nowrap;">
            <b>{place["こども食堂の名称"]}</b><br>
            施設名：{place["施設名"]}<br>
            開催日：{place["開催日"]}<br>
            時間：{place["時間"]}<br>
            参加費用：{place["参加費用"]}<br>
            電話番号：{place["電話番号"]}<br>
            その他：{place["その他"]}
        </div>
        """
        #営業日かどうかでアイコンの色を変更
        if place.get("営業中", False):
            # 赤で大きめ
            icon = CustomIcon(
                icon_image='https://raw.githubusercontent.com/pointhi/leaflet-color-markers/master/img/marker-icon-red.png',
                icon_size=(40, 65)  # サイズ大きめ
            )
        else:
            # 青で標準サイズ
            icon = CustomIcon(
                icon_image='https://raw.githubusercontent.com/pointhi/leaflet-color-markers/master/img/marker-icon-blue.png',
                icon_size=(25, 41)  # デフォルト標準サイズ
            )
            
        folium.Marker(
            location = [place["lat"], place["long"]],
            popup = popup_html,
            icon = icon
    ).add_to(m)

# HTMLファイルとして保存
m.save("map.html")
print("map.html をブラウザで開いてください")

map.html をブラウザで開いてください


In [24]:
import folium
from folium.features import CustomIcon
import json
import pandas as pd
import datetime

# サンプル df（実際はあなたの df を使用）
# 'lat', 'long', 'こども食堂の名称', '日時' 列が必須
# df = pd.read_csv("your_data.csv")

# 地図の中心
chiba_center = [35.6076, 140.1063]
sw = [35.50, 140.05]
ne = [35.70, 140.20]

# Folium マップ
m = folium.Map(
    location=chiba_center,
    zoom_start=13,
    tiles='https://cyberjapandata.gsi.go.jp/xyz/pale/{z}/{x}/{y}.png',
    attr='国土地理院',
    min_zoom=12,
    max_zoom=18,
    max_bounds=True
)
m.fit_bounds([sw, ne])

# JS に渡す用のデータを作成
places_data = df[['lat','long','こども食堂の名称','日時']].to_dict(orient='records')

# カレンダーと JS を埋め込む
m.get_root().html.add_child(folium.Element(f"""
<div style="position: fixed; top: 10px; left: 50px; z-index: 9999; background: white; padding: 5px;">
    <label>日付選択: </label>
    <input type="date" id="dateSelect" value="{datetime.date.today()}">
</div>

<script>
var places = {json.dumps(places_data)};

// アイコン定義
var redIcon = new L.Icon({{iconUrl:'https://raw.githubusercontent.com/pointhi/leaflet-color-markers/master/img/marker-icon-red.png',iconSize:[40,65]}});
var blueIcon = new L.Icon({{iconUrl:'https://raw.githubusercontent.com/pointhi/leaflet-color-markers/master/img/marker-icon-blue.png',iconSize:[25,41]}});

// マーカー作成
places.forEach(function(p){{
    p.marker = L.marker([p.lat, p.long], {{icon: blueIcon}}).addTo(map)
        .bindPopup(p.こども食堂の名称);
}});

// 営業判定関数
function getWeekCode(date){{
    let weekday = date.getDay();
    if(weekday==0) weekday=7;
    let week_num = Math.floor((date.getDate()-1)/7)+1;
    let code = (week_num-1)*7 + weekday;
    return code.toString().padStart(2,"0");
}}

function openToday(codeStr, date){{
    if(!codeStr) return false;
    if(codeStr.includes("EV") && (date.getMonth()+1)%2==1) return false;
    let todayCode = getWeekCode(date);
    let codes = [];
    for(let i=0;i<codeStr.length;i+=2) codes.push(codeStr.substring(i,i+2));
    return codes.includes(todayCode);
}}

// 日付変更時にマーカーを更新
function updateMarkers(){{
    let input = document.getElementById("dateSelect").value;
    if(!input) return;
    let date = new Date(input);
    places.forEach(function(p){{
        if(openToday(p.日時, date)) p.marker.setIcon(redIcon);
        else p.marker.setIcon(blueIcon);
    }});
}}

document.getElementById("dateSelect").addEventListener("change", updateMarkers);
updateMarkers();
</script>
"""))


# HTML 保存
m.save("map.html")
print("map.html をブラウザで開いてください")


map.html をブラウザで開いてください


In [73]:
# 今日の日付（デバッグ用に固定したいなら date(2025,9,6) みたいに指定可能）
import datetime
today = datetime.date(2025,9,28)

# "開催日" から判定して新しい列 "営業中" に結果を入れる
df["営業中"] = df["開催日"].apply(lambda x: open_today(str(x), today))

# 結果を確認
print(df[["こども食堂の名称", "開催日", "営業中"]])
print(today)

第３土曜日
第2木曜日夕方第4日曜日昼
第４日曜日
            こども食堂の名称                              開催日    営業中
0          きぼーるこども食堂        毎月1回（曜日は未定）6月～9月は開催いたしません  False
1        こども食堂 やまちゃん                     夏休み限定（週1～2回）  False
2      グレイステーブルこども食堂           毎月1回土曜日学校の長期休暇期間は不定期開催  False
3           松ケ丘子ども食堂                              年数回  False
4              こども食堂                          毎月第1火曜日  False
5          ラヴィーレ千葉椿森                          毎月第4土曜日  False
6        松波フレンズこども食堂                            第３土曜日  False
7           幸福軒子ども食堂                          毎月第３木曜日  False
8       おおなみこなみ「青空市」                            年３回程度  False
9          西小中台子ども食堂                          毎月第4土曜日  False
10          和らぎ子ども食堂                   毎月第2日曜日（休日開庁日）  False
11            はたまち食堂                          毎月第3金曜日  False
12           柏井こども食堂                    第2木曜日夕方第4日曜日昼  False
13         そんぽの家朝日ヶ丘                          毎月第4土曜日  False
14            あったカフェ                          毎月第１土曜日  Fals

In [55]:
import datetime
import re
datetime.date.today().weekday()

6

In [29]:
import folium
from folium.plugins import MousePosition

# 地図を作成（中心位置は千葉市）
m = folium.Map(location=[35.6073, 140.1063], zoom_start=12)

# マウスの位置情報を表示する
MousePosition(
    position='topright', 
    separator=' : ',
    prefix='緯度経度',
    lat_formatter="function(num) {return num.toFixed(6);}",
    lng_formatter="function(num) {return num.toFixed(6);}"
).add_to(m)

# 地図表示
m


In [20]:
import folium
from folium.plugins import MarkerCluster
import geopandas as gpd
from geopy.geocoders import Nominatim
import time

# 住所リスト
places = [
    {"name": "千葉市中央区役所", "address": "千葉市中央区中央", "info": "中央区役所です"},
    {"name": "千葉駅", "address": "千葉市中央区新千葉", "info": "千葉駅です"},
]

# ジオコーダー
geolocator = Nominatim(user_agent="my_map_app")

# 千葉市中心
chiba_center = [35.6076, 140.1063]

# 千葉市境界 GeoJSON/ Shapefile を読み込み
gdf = gpd.read_file("chiba_city_boundary.shp")  # 千葉市の境界データ

# 千葉市の境界座標
chiba_bounds = gdf.total_bounds  # [minx, miny, maxx, maxy]
sw = [chiba_bounds[1], chiba_bounds[0]]  # 南西
ne = [chiba_bounds[3], chiba_bounds[2]]  # 北東

# 地図作成（白地図）
m = folium.Map(
    location=chiba_center,
    zoom_start=13,
    tiles='CartoDB Positron',
    min_zoom=12,
    max_zoom=18,
    max_bounds=True
)

# 千葉市境界ポリゴンを描画（内部は透明で境界線のみ）
folium.GeoJson(
    gdf,
    style_function=lambda x: {
        'fillColor': 'white',
        'color': 'black',
        'weight': 2,
        'fillOpacity': 0
    }
).add_to(m)

# 千葉市外を半透明で覆うマスク
folium.Rectangle(
    bounds=[[-90, -180], [90, 180]],  # 世界全体
    color='gray',
    fill=True,
    fill_opacity=0.5
).add_to(m)

# MarkerClusterでピンを整理
marker_cluster = MarkerCluster().add_to(m)

# ピン追加（千葉市内のみ）
for place in places:
    loc = geolocator.geocode(place['address'])
    if loc is None:
        print(f"住所が見つかりません: {place['address']}")
        continue
    lat, lng = loc.latitude, loc.longitude
    if not (sw[0] <= lat <= ne[0] and sw[1] <= lng <= ne[1]):
        continue
    folium.Marker(
        [lat, lng],
        popup=f"<b>{place['name']}</b><br>{place['info']}",
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(marker_cluster)
    time.sleep(1)  # geopy制限対策

# 保存
m.save("chiba_cutout_map.html")
print("chiba_cutout_map.html をブラウザで開いてください")


DataSourceError: chiba_city_boundary.shp: No such file or directory

In [25]:
import pandas as pd
import json

# dfはすでに読み込まれていると仮定
# df = pd.read_csv(...)

# '日時'が欠損値の場合を考慮して、文字列に変換
df['日時'] = df['日時'].fillna('')

# JSONファイルとして保存するデータフレームを作成
# '営業中'の列はブラウザ側で判定するため不要
df_to_json = df[['こども食堂の名称', '施設名', '開催日', '時間', '参加費用', '電話番号', 'その他', 'lat', 'long', '日時']]

# 必要な列をリストに変換
data_list = df_to_json.to_dict('records')

# JSONファイルとして保存
with open('kodomo_shokudo_data.json', 'w', encoding='utf-8') as f:
    json.dump(data_list, f, ensure_ascii=False, indent=4)

print("kodomo_shokudo_data.jsonが作成されました。")

kodomo_shokudo_data.jsonが作成されました。


IndentationError: unindent does not match any outer indentation level (<string>, line 13)